# Getting started with DoWhy: A simple example
This is quick introduction to DoWhy causal inference library.
We will load in a sample dataset and estimate causal effect from a (pre-specified)treatment variable to a (pre-specified) outcome variable.

First, let us add required path for python to find DoWhy code and load required packages.

In [1]:
import os, sys
sys.path.append(os.path.abspath("../../"))

In [2]:
import numpy as np
import pandas as pd

import dowhy
from dowhy.do_why import CausalModel
import dowhy.datasets 

Let us first load a dataset. For simplicity, we simulate a dataset with linear relationships between common causes and treatment, and common causes and outcome. 

Beta is the true causal effect. 

In [3]:
data = dowhy.datasets.linear_dataset(beta=10,
        num_common_causes=5,
        num_instruments = 2,
        num_samples=10000, 
        treatment_is_binary=True)
df = data["df"]
print(df.head())
print(data["dot_graph"])

    Z0        Z1        X0        X1        X2        X3        X4    v  \
0  0.0  0.682129  0.619275 -1.338917 -1.344954 -1.205516 -0.380428  0.0   
1  0.0  0.780360  1.489020  0.224417 -0.925444 -1.178246 -1.462796  1.0   
2  0.0  0.749278  0.338573  0.200353  0.144210  0.006207 -0.311147  1.0   
3  0.0  0.604209  0.104159 -0.659282  0.186016 -0.054680 -0.065804  1.0   
4  0.0  0.966614  0.441749 -1.867424 -0.851817 -0.843792 -1.123326  0.0   

           y  
0  -9.677161  
1   4.996252  
2  11.352700  
3   9.375556  
4 -10.086670  
graph[node[ id "v" label "v"]node[ id "y" label "y"]node[ id "Unobserved Confounders" label "Unobserved Confounders"]edge[source "v" target "y"]edge[source "Unobserved Confounders" target "v"]edge[source "Unobserved Confounders" target "y"]node[ id "X0" label "X0"] edge[ source "X0" target "v"] node[ id "X1" label "X1"] edge[ source "X1" target "v"] node[ id "X2" label "X2"] edge[ source "X2" target "v"] node[ id "X3" label "X3"] edge[ source "X3" target 

Note that we are using a pandas dataframe to load the data. At present, DoWhy only supports pandas dataframe as input.

## Interface 1 (recommended): Input causal graph

We now input a causal graph in the DOT graph format.

In [4]:
# With graph
model=CausalModel(
        data = df,
        treatment=data["treatment_name"],
        outcome=data["outcome_name"],
        graph=data["dot_graph"],
        )

Model to find the causal effect of treatment v on outcome y


In [5]:
model.view_model()

<img src="causal_model.png">

We get the same causal graph. Now identification and estimation is done as before.

In [6]:
identified_estimand = model.identify_effect()
print(identified_estimand)

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:{'X3', 'Z0', 'X4', 'Z1', 'Unobserved Confounders', 'X2', 'X0', 'X1'}


{'observed': 'yes'}
{'observed': 'yes'}
{'observed': 'yes'}
{'observed': 'yes'}
{'observed': 'no'}
There are unobserved common causes. Causal effect cannot be identified.
WARN: Do you want to continue by ignoring these unobserved confounders? [y/n] y


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]


Estimand type: ate
### Estimand : 1
Estimand name: iv
No such variable found!
### Estimand : 2
Estimand name: backdoor
Estimand expression:
d                                      
──(Expectation(y|X3,Z0,X4,Z1,X2,X0,X1))
dv                                     
Estimand assumption 1, Unconfoundedness: If U→v and U→y then P(y|v,X3,Z0,X4,Z1,X2,X0,X1,U) = P(y|v,X3,Z0,X4,Z1,X2,X0,X1)



In [7]:
causal_estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))

LinearRegressionEstimator


INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: y~v+X3+Z0+X4+Z1+X2+X0+X1


*** Causal Estimate ***

## Target estimand
Estimand type: ate
### Estimand : 1
Estimand name: iv
No such variable found!
### Estimand : 2
Estimand name: backdoor
Estimand expression:
d                                      
──(Expectation(y|X3,Z0,X4,Z1,X2,X0,X1))
dv                                     
Estimand assumption 1, Unconfoundedness: If U→v and U→y then P(y|v,X3,Z0,X4,Z1,X2,X0,X1,U) = P(y|v,X3,Z0,X4,Z1,X2,X0,X1)

## Realized estimand
b: y~v+X3+Z0+X4+Z1+X2+X0+X1
## Estimate
Value: 9.999999999999842

Causal Estimate is 10.0


## Interface 2: Specify common causes and instruments

In [8]:
# Without graph                                       
model= CausalModel(                             
        data=df,                                      
        treatment=data["treatment_name"],             
        outcome=data["outcome_name"],                 
        common_causes=data["common_causes_names"])                         

Model to find the causal effect of treatment v on outcome y


In [9]:
model.view_model()

<img src="causal_model.png" />

The above causal graph shows the assumptions encoded in the cauasl model. We can now use this graph to first identify 
the causal effect (go from a causal estimand to a probability expression), and then estimate the causal effect.

**DoWhy philosophy: Keep identification and estimation separate**

Identification can be achieved without access to data, only the graph. This results in an expression to computed. This expression can then be computed using the available data in the estimation step.
Important to understand that these are orthogonal steps.

* Identification

In [10]:
identified_estimand = model.identify_effect()                         

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:{'X3', 'X4', 'X2', 'X0', 'X1', 'U'}


{'observed': 'yes'}
{'observed': 'yes'}
{'observed': 'yes'}
{'observed': 'yes'}
{'observed': 'yes'}
{'label': 'Unobserved Confounders', 'observed': 'no'}
There are unobserved common causes. Causal effect cannot be identified.
WARN: Do you want to continue by ignoring these unobserved confounders? [y/n] y


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]


* Estimation

In [11]:
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression", 
        test_significance=True
        )         
print(estimate)
print("Causal Estimate is " + str(estimate.value))

INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: y~v+X3+X4+X2+X0+X1


LinearRegressionEstimator
*** Causal Estimate ***

## Target estimand
Estimand type: ate
### Estimand : 1
Estimand name: iv
No such variable found!
### Estimand : 2
Estimand name: backdoor
Estimand expression:
d                                
──(Expectation(y|X3,X4,X2,X0,X1))
dv                               
Estimand assumption 1, Unconfoundedness: If U→v and U→y then P(y|v,X3,X4,X2,X0,X1,U) = P(y|v,X3,X4,X2,X0,X1)

## Realized estimand
b: y~v+X3+X4+X2+X0+X1
## Estimate
Value: 9.999999999999849

## Statistical Significance
p-value: 0.0

Causal Estimate is 10.0


## Refuting the estimate

Now refuting the obtained estimate.

### Adding a random common cause variable

In [12]:
res_random=model.refute_estimate(identified_estimand, estimate, method_name="random_common_cause")
print(res_random)

INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: y~v+X3+X4+X2+X0+X1+w_random


Refute: Add a Random Common Cause
Estimated effect:(9.999999999999849,)
New effect:(9.9999999999998561,)



### Replacing treatment with a random (placebo) variable

In [13]:
res_placebo=model.refute_estimate(identified_estimand, estimate,
        method_name="placebo_treatment_refuter", placebo_type="permute")
print(res_placebo)

INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: y~placebo+X3+X4+X2+X0+X1


Refute: Use a Placebo Treatment
Estimated effect:(9.999999999999849,)
New effect:(-0.039298742866320742,)



### Removing a random subset of the data

In [14]:
res_subset=model.refute_estimate(identified_estimand, estimate,
        method_name="data_subset_refuter", subset_fraction=0.9)
print(res_subset)


INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: y~v+X3+X4+X2+X0+X1


Refute: Use a subset of data
Estimated effect:(9.999999999999849,)
New effect:(9.9999999999998206,)



As you can see, the linear regression estimator is very sensitive to simple refutations.